**بناء روبوت دردشة مفتوح المصدر بستخدام نموذج (Blenderbot) للاجابة علي الاسئلة**

## **Impoer required libraries**

In [34]:
import torch
import transformers
from transformers import pipeline
from datasets import load_dataset

In [2]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

In [ ]:
! pip install datasets

## **App 1**

**Load Model**--->**Using pipeline**

In [ ]:
# Use a pipeline as a high-level helper
model = pipeline("text2text-generation", model="facebook/blenderbot-400M-distill")

In [ ]:
user_prompt = "what is the best ways to learn Physics)"
response = model(user_prompt)

# Print the response
print(response[0]['generated_text'])

 The best way to learn physics is to understand how the universe works and how it functions.


**Call function**
* بنعمل الدالة دي عشان نحافظ سجل المحادثات زي الذاكرة في لانجتشين
* نقدر بعدها نكمل علي الرسايل القديمة

In [ ]:
def add_message_and_response(history, message):

  history.append({"role": "user", "content": message}) # لحفظ سجل المحادثات

  ## Combine all previous messages for context
  full_conversation = " ".join([msg["content"] for msg in history])
  response = model(full_conversation)

  history.append({"role": "bot", "content": response[0]['generated_text']})

  return response[0]['generated_text']

In [ ]:
# Start the conversation
conversation_history = []

prompt = "What are the best ways to learn AI?"
response = add_message_and_response(conversation_history, prompt)

# Print the response
print("Bot:", response)

Bot:  The best way to learn about it is by watching videos on the internet. There are so many ways to do it.


**Load Model using directly**

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/blenderbot-400M-distill")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/blenderbot-400M-distill")

# **App 2**

**Audio classification**

**Load the Dataset and Model**

In [ ]:
ds = load_dataset("ashraq/esc50", split="train[0:20]") # عند كتابة التقسيم نأخذ الداتا المقسمة فقط

In [35]:
sample = ds[15] # shown sample of audios
sample

{'filename': '1-110537-A-22.wav',
 'fold': 1,
 'target': 22,
 'category': 'clapping',
 'esc10': False,
 'src_file': 110537,
 'take': 'A',
 'audio': {'path': None,
  'array': array([-0.24393228, -0.27682328, -0.21330422, ..., -0.71662188,
         -0.65534687,  0.        ]),
  'sampling_rate': 48000}}

**Shown some audio**

In [33]:
from IPython.display import Audio
Audio(sample["audio"]["array"], rate = sample["audio"]["sampling_rate"])

**Load the Model**

In [16]:
pipe = pipeline("zero-shot-audio-classification", model="laion/clap-htsat-unfused")

Device set to use cuda:0


In [17]:
pipe.feature_extractor.sampling_rate # عرض عدد العينات

48000

غير متوافق مع عدد بيانات التدريب؟؟؟؟؟ -  'sampling_rate': 44100' of data

In [18]:
from datasets import Audio
ds = ds.cast_column("audio", Audio(sampling_rate = pipe.feature_extractor.sampling_rate))

In [29]:
sample = ds[15] # audio number n
sample

{'filename': '1-110537-A-22.wav',
 'fold': 1,
 'target': 22,
 'category': 'clapping',
 'esc10': False,
 'src_file': 110537,
 'take': 'A',
 'audio': {'path': None,
  'array': array([-0.24393228, -0.27682328, -0.21330422, ..., -0.71662188,
         -0.65534687,  0.        ]),
  'sampling_rate': 48000}}

**classification**

In [25]:
labels = ['dog', 'cleaner', 'lion','cat', 'dor','clapping']
pipe(sample["audio"]["array"], candidate_labels = labels)

[{'score': 0.9972768425941467, 'label': 'clapping'},
 {'score': 0.00264929560944438, 'label': 'dor'},
 {'score': 5.256431904854253e-05, 'label': 'cleaner'},
 {'score': 9.794497600523755e-06, 'label': 'dog'},
 {'score': 6.6811971919378266e-06, 'label': 'cat'},
 {'score': 4.819786227017175e-06, 'label': 'lion'}]

#**App 3**